In [1]:

import json
from openai import OpenAI
from dataclasses import dataclass, field
from typing import List, Callable

In [84]:
from dataclasses import dataclass, field
from typing import Callable, List
from pprint import pprint
import json
from openai import OpenAI  # Make sure you have the correct import
from rich.console import Console
import inspect

# Setup OpenAI client to use Ollama
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)


@dataclass
class Tool:
    """Tool class similar to agents.Tool"""
    name: str
    description: str
    function: Callable
    parameters: dict


@dataclass
class Agent:
    """Agent class with tools"""
    name: str
    instructions: str
    tools: List[Tool] = field(default_factory=list)
    model: str = "llama3.2:1b"  # Make sure the model name matches Ollama's available models


class Runner:
    """Runner class for executing agents"""

    @classmethod
    def run_sync(cls, agent: Agent, input: str, max_turns: int = 5) -> str:
        """Run agent synchronously"""
        messages = [
            {"role": "system", "content": agent.instructions},
            {"role": "user", "content": input}
        ]

        tools_schema = [tool.parameters for tool in agent.tools] if agent.tools else None

        while True:
            pprint(messages)
            response = client.chat.completions.create(
                model=agent.model,
                messages=messages,
                tools=tools_schema
            )

            message = response.choices[0].message

            # If the model calls a tool
            if message.tool_calls:
                messages.append({
                    "role": "assistant",
                    "content": message.content,
                    "tool_calls": [
                        {
                            "id": tc.id,
                            "type": tc.type,
                            "function": {
                                "name": tc.function.name,
                                "arguments": tc.function.arguments
                            }
                        }
                        for tc in message.tool_calls
                    ]
                })

                # Execute tool calls
                for tool_call in message.tool_calls:
                    function_name = tool_call.function.name
                    arguments = json.loads(tool_call.function.arguments)

                    result = "Tool not found"
                    for tool in agent.tools:
                        if tool.name == function_name:
                            try:
                                result = tool.function(**arguments)
                                print("==" * 20, result)
                            except Exception as e:
                                result = f"Error: {e}"
                            break

                    messages.append({
                        "role": "tool",
                        "content": str(result),
                        "tool_call_id": tool_call.id
                    })

            elif response.choices[0].finish_reason == "stop":
                Console().print(f"[green]{message.content}[/green]")
                return message.content

            else:
                # In rare cases, you may need to continue collecting response chunks
                Console().print("[yellow]Continuing conversation...[/yellow]")
                messages.append({
                    "role": "assistant",
                    "content": message.content
                })


def agent_as_tool(agent: Agent) -> Tool:
    """Convert an agent into a tool that can be used by other agents"""

    def agent_function(query: str) -> str:
        """Execute the agent with the given query"""
        print(f"  🤖 Calling {agent.name} agent...")
        result = Runner.run_sync(agent, query, max_turns=3)
        return result

    # Create tool schema
    schema = {
        "type": "function",
        "function": {
            "name": f"use_{agent.name.lower().replace(' ', '_')}_agent",
            "description": f"Use {agent.name} to handle: {agent.instructions[:100]}...",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to send to the agent"
                    }
                },
                "required": ["query"]
            }
        }
    }

    return Tool(
        name=f"use_{agent.name.lower().replace(' ', '_')}_agent",
        description=f"Tool wrapper for agent: {agent.name}",
        function=agent_function,
        parameters=schema
    )


def function_tool(func: Callable) -> Tool:
    """Decorator similar to agents.function_tool"""

    # Get function signature
    sig = inspect.signature(func)
    properties = {}
    required = []

    for param_name, param in sig.parameters.items():
        properties[param_name] = {
            "type": "string",
            "description": f"Parameter {param_name}"
        }
        if param.default == inspect.Parameter.empty:
            required.append(param_name)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": func.__doc__ or f"Function {func.__name__}",
            "parameters": {
                "type": "object",
                "properties": properties,
                "required": required
            }
        }
    }

    return Tool(
        name=func.__name__,
        description=func.__doc__ or f"Function {func.__name__}",
        function=func,
        parameters=schema
    )


In [85]:
weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a weather specialist. Only answer weather questions. Be very concise."
)

math_agent = Agent(
    name="Math Agent",
    instructions="You are a math specialist. Only solve math problems. Show brief work."
)

research_agent = Agent(
    name="Research Agent",
    instructions="You are a research specialist. Only answer research questions. Be informative but concise."
)
    

In [86]:
from pprint import pprint
pprint(agent_as_tool(weather_agent))

Tool(name='use_weather_agent_agent',
     description='Tool wrapper for agent: Weather Agent',
     function=<function agent_as_tool.<locals>.agent_function at 0x71fc3a608860>,
     parameters={'function': {'description': 'Use Weather Agent to handle: You '
                                             'are a weather specialist. Only '
                                             'answer weather questions. Be '
                                             'very concise....',
                              'name': 'use_weather_agent_agent',
                              'parameters': {'properties': {'query': {'description': 'Query '
                                                                                     'to '
                                                                                     'send '
                                                                                     'to '
                                                                                     't

In [87]:

# Create supervisor agent that uses other agents as tools
supervisor_agent = Agent(
    name="Supervisor Agent",
    instructions="""You are a supervisor that delegates work to specialist agents.
    Use the appropriate agent tool for different types of queries:
    - use_weather_agent_agent for weather questions
    - use_math_agent_agent for math problems
    - use_research_agent_agent for research questions
    Answer simple greetings directly without using tools.""",
    tools=[
        agent_as_tool(weather_agent),
        agent_as_tool(math_agent),
        agent_as_tool(research_agent)
    ]
)

In [88]:
query = "What's the weather in Paris and what's 50 divided by 2?"
print(Runner.run_sync(supervisor_agent, query,max_turns=5))

[{'content': 'You are a supervisor that delegates work to specialist agents.\n'
             '    Use the appropriate agent tool for different types of '
             'queries:\n'
             '    - use_weather_agent_agent for weather questions\n'
             '    - use_math_agent_agent for math problems\n'
             '    - use_research_agent_agent for research questions\n'
             '    Answer simple greetings directly without using tools.',
  'role': 'system'},
 {'content': "What's the weather in Paris and what's 50 divided by 2?",
  'role': 'user'}]
  🤖 Calling Weather Agent agent...
[{'content': 'You are a weather specialist. Only answer weather questions. Be '
             'very concise.',
  'role': 'system'},
 {'content': 'What is the weather in Paris?', 'role': 'user'}]


It currently snows heavily in Paris during this month.

======================================== It currently snows heavily in Paris during this month.
  🤖 Calling Math Agent agent...
[{'content': 'You are a math specialist. Only solve math problems. Show brief '
             'work.',
  'role': 'system'},
 {'content': '50 / 2', 'role': 'user'}]


50 ÷ 2 = 25

======================================== 50 ÷ 2 = 25
[{'content': 'You are a supervisor that delegates work to specialist agents.\n'
             '    Use the appropriate agent tool for different types of '
             'queries:\n'
             '    - use_weather_agent_agent for weather questions\n'
             '    - use_math_agent_agent for math problems\n'
             '    - use_research_agent_agent for research questions\n'
             '    Answer simple greetings directly without using tools.',
  'role': 'system'},
 {'content': "What's the weather in Paris and what's 50 divided by 2?",
  'role': 'user'},
 {'content': '} \n'
             '\n'
             '[\n'
             '    {\n'
             '        "name": "use_weather_agent_agent",\n'
             '        "parameters": {}\n'
             '    },\n'
             '    {\n'
             '        "name": "use_math_agent_agent",\n'
             '        "parameters": {}\n'
             '    }\n'
             ']',
  'role': 

I'm glad I could help with the weather and math queries. If you have any more questions or need assistance with 
anything else, feel free to ask!

I'm glad I could help with the weather and math queries. If you have any more questions or need assistance with anything else, feel free to ask!
